In [1]:
import torch
print(torch.__version__)
import monai
print(monai.__version__)
import pandas 
print(pandas.__version__)

2.0.0+cu117
1.1.0
1.1.5


In [2]:
#modified from 240714_main_(2D_add_slice).ipynb
import numpy as np
import pandas as pd
import time
import timeit
from datetime import datetime
import os
import glob
import natsort
import sys
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
import cv2
from PIL import Image, ImageDraw
import json
import random
import copy
import warnings
warnings.filterwarnings('ignore')
# import ipynbname
if '__file__' in globals():
    FILENAME = os.getcwd() + '/' + os.path.basename(__file__)
else:
    try:
        from ipynbname import name
        FILENAME = os.getcwd() + '/' + name() + '.ipynb'
    except ModuleNotFoundError:
        FILENAME = os.getcwd() + '/notebook_name.ipynb'

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import RandomResizedCrop
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split, KFold

from monai.losses import TverskyLoss as TverskyLoss
from monai.transforms import Compose, ToTensor, RandFlip
from monai.metrics import DiceMetric as Dice_Function
from monai.metrics import compute_iou as IoU_Function
from monai.metrics import ConfusionMatrixMetric

from torchvision.transforms import InterpolationMode

import sys
sys.path.append("..")

model_dir = 'models'
# module_names = [py.replace('.py','') for py in os.listdir(model_dir)] ; module_names = list(set(module_names)-{'.ipynb_checkpoints','__pycache__'});
module_names = ['DeepLab_V3_Plus_Effi_U_Trans2']
Dataset_dir = 'CarDD_release/splits'
 
model_names = module_names

for  module_name in module_names:
    exec(f'from {model_dir}.{module_name} import *')
# dataset_path-training
#         ㄴ V01_01_002.jpg, V01_01_002_mask.jpg, ...
# ㄴvalidation
#  ㄴtest

iterations = [1, 10]
# train_size=0.6

in_channels = 3
number_of_classes=1
epochs = 100 # 125
EARLY_STOP = 25  #25
batch_size = 8

devices = [0]

optimizer = 'AdamW'
lr = 1e-4
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}

lr_scheduler = 'CosineAnnealingLR'
T_max = epochs
T_0 = epochs
eta_min = 1e-6
lr_scheduler_args = {'lr_scheduler': lr_scheduler, 'T_max': T_max, 'T_0': T_0, 'eta_min': eta_min}

loss_function = 'DiceBCELoss'
#loss_function = 'DiceBCELoss'
# loss_function = 'Tversky Focal Loss'
reduction = 'mean'
gamma = 2.0
weight = None
loss_function_args = {'loss_function': loss_function, 'reduction': reduction, 'gamma': gamma, 'weight': weight}

In [3]:
def control_random_seed(seed, pytorch=True):
    random.seed(seed)
    np.random.seed(seed)
    try:
        torch.manual_seed(seed)
        if torch.cuda.is_available()==True:
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
    except:
        pass
        torch.backends.cudnn.benchmark = False
def imread_kor ( filePath, mode=cv2.IMREAD_UNCHANGED ) : 
    stream = open( filePath.encode("utf-8") , "rb") 
    bytes = bytearray(stream.read()) 
    numpyArray = np.asarray(bytes, dtype=np.uint8)
    return cv2.imdecode(numpyArray , mode)
def imwrite_kor(filename, img, params=None): 
    try: 
        ext = os.path.splitext(filename)[1] 
        result, n = cv2.imencode(ext, img, params) 
        if result:
            with open(filename, mode='w+b') as f: 
                n.tofile(f) 
                return True
        else: 
            return False 
    except Exception as e: 
        print(e) 
        return False
    
def random_rotation(image, mask, angle_range=(-30, 30)):
    # 지정된 각도 범위 내에서 무작위로 각도 선택
    angle = random.uniform(angle_range[0], angle_range[1])
    # 이미지와 마스크를 동일한 각도로 회전
    image = TF.rotate(image, angle)
    mask = TF.rotate(mask, angle)
    return image, mask

def load_coco_data(base_dataset_dir, sub_dirs=['training', 'validation', 'test']):
    data = {}
    for sub_dir in sub_dirs:
        # 각 폴더의 JSON 파일 경로
        json_path = os.path.join(base_dataset_dir, sub_dir, f"{sub_dir}_data.json")
        
        # JSON 파일 로드
        with open(json_path, 'r') as f:
            coco_data = json.load(f)
        
        # 이미지와 주석 매핑
        images = {img["id"]: img for img in coco_data["images"]}
        annotations = {}
        for ann in coco_data["annotations"]:
            image_id = ann["image_id"]
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(ann)
        
        data[sub_dir] = {
            "images": images,
            "annotations": annotations
        }
    
    return data

def create_mask(image_size, annotations):
    """
    COCO annotation의 segmentation 정보를 활용해 마스크 생성
    """
    mask = Image.new("L", (image_size[1], image_size[0]), 0)  # L = grayscale
    draw = ImageDraw.Draw(mask)
    
    for ann in annotations:
        segmentation = ann["segmentation"]
        for polygon in segmentation:
            draw.polygon(polygon, outline=1, fill=1)
    
    return np.array(mask)

class ImagesDataset(Dataset):
    def __init__(self, images, annotations, image_dir, aug=False):
        self.images = list(images.values())  # 이미지 정보를 리스트로 변환
        self.annotations = annotations
        self.image_dir = image_dir
        self.aug = aug
        self.desired_height = 256
        self.desired_width = 256

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_info = self.images[idx]
        image_id = image_info["id"]
        image_path = os.path.join(self.image_dir, image_info["file_name"])
        
        # 이미지 로드
        image = Image.open(image_path).convert("RGB")
        image = image.resize((self.desired_width, self.desired_height), resample=Image.BILINEAR)
        
        # 마스크 생성
        annotations = self.annotations.get(image_id, [])
        mask = create_mask((image_info["height"], image_info["width"]), annotations)
        mask = Image.fromarray(mask).resize((self.desired_width, self.desired_height), resample=Image.NEAREST)
        
        # 데이터 증강
        if self.aug:
            if random.random() < 0.5:
                image = TF.hflip(image)
                mask = TF.hflip(mask)
            if random.random() < 0.5:
                angle = random.uniform(-30, 30)
                image = TF.rotate(image, angle, interpolation=InterpolationMode.BILINEAR)
                mask = TF.rotate(mask, angle, interpolation=InterpolationMode.NEAREST)
        
        # 텐서로 변환
        image = TF.to_tensor(image).float()
        mask = torch.from_numpy(np.array(mask)).float()
        mask[mask > 0] = 1.0
        if mask.dim() == 2:
            mask = mask.unsqueeze(0)
        
        return image, mask, image_path

def create_dataloader(coco_data, sub_dir, image_dir, batch_size, aug=False):
    dataset = ImagesDataset(
        images=coco_data[sub_dir]["images"],
        annotations=coco_data[sub_dir]["annotations"],
        image_dir=image_dir,
        aug=aug
    )
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=(sub_dir == "training"),
        num_workers=4, pin_memory=True
    )
    return dataloader

def Pixel_Accuracy(yhat, ytrue, threshold=0.5):
    yhat = yhat>threshold
    if ytrue.dim() == 3:
        ytrue = ytrue.unsqueeze(1)
    correct = torch.sum(yhat == ytrue)
    total = ytrue.numel()
    accuracy = correct.float() / total
    return accuracy.item()

def Intersection_over_Union(yhat, ytrue, threshold=0.5):
    yhat = yhat>threshold
    if ytrue.dim() == 3:
        ytrue = ytrue.unsqueeze(1)
    return IoU_Function(yhat, ytrue).nanmean().item()
 
def Dice_Coefficient(yhat, ytrue, threshold=0.5):
    yhat = yhat>threshold
    if ytrue.dim() == 3:
        ytrue = ytrue.unsqueeze(1)
    return Dice_Function()(yhat, ytrue).nanmean().item()

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # 시그모이드 함수 적용하여 예측 확률 계산
        inputs_sigmoid = torch.sigmoid(inputs)
        
        intersection = (inputs_sigmoid * targets).sum()                           
        dice_loss = 1 - (2. * intersection + smooth) / (inputs_sigmoid.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy_with_logits(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE

class DiceCELoss(nn.Module):
    def __init__(self, weight=None, smooth=1.0):
        """
        Dice + Cross-Entropy Loss
        Args:
            weight (list or None): 각 클래스의 가중치 (Cross-Entropy에서 사용)
            smooth (float): Dice 손실 계산 시 안정성을 위한 smoothing 값
        """
        super(DiceCELoss, self).__init__()
        self.weight = weight
        self.smooth = smooth

    def forward(self, inputs, targets):
        """
        Args:
            inputs (torch.Tensor): 모델의 출력 (logits, shape: [batch, num_classes, height, width])
            targets (torch.Tensor): 정답 레이블 (shape: [batch, height, width] 또는 [batch, num_classes, height, width])
        Returns:
            torch.Tensor: Dice + Cross-Entropy 손실 값
        """
        # 예측 결과를 확률로 변환
        inputs = torch.softmax(inputs, dim=1)
        
        # Dice Loss 계산
        targets_one_hot = F.one_hot(targets, num_classes=inputs.size(1)).permute(0, 3, 1, 2).float()
        intersection = (inputs * targets_one_hot).sum(dim=(2, 3))
        dice_loss = 1 - (2. * intersection + self.smooth) / (inputs.sum(dim=(2, 3)) + targets_one_hot.sum(dim=(2, 3)) + self.smooth)
        dice_loss = dice_loss.mean()  # 배치 평균

        # Cross-Entropy Loss 계산
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction='mean')

        return dice_loss + ce_loss
    
def Confusion_Matrix(yhat, ytrue, threshold=0.5):
    yhat = yhat>threshold
    confusion_matrix = ConfusionMatrixMetric(metric_name = ["recall", "precision", "f1 score"], reduction ='mean', compute_sample =True)
    confusion_matrix(yhat, ytrue)
    recall, precision, f1 = confusion_matrix.aggregate()
    return recall, precision, f1
    
def train(train_loader, epoch, \
          model, criterion, optimizer, device
          ):
    model.train()
    train_losses=AverageMeter()
    for i, (input, target, _) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        output = model(input)

        # target이 output과 동일한 크기를 갖도록 함
        if target.dim() == 3:
            target = target.unsqueeze(1)

        loss = criterion(output,target).float()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.update(loss.detach().cpu().numpy(),input.shape[0])
    Train_Loss=np.round(train_losses.avg,6)
    return Train_Loss
def validate(validation_loader, 
          model, criterion, device,
        model_path=False,
             return_image_paths=False,
          ):
    if model_path!=False:
        model.load_state_dict(torch.load(model_path))
    model.eval()
    for i, (input, target, image_path) in enumerate(validation_loader):
        input = input.to(device)
        target = target.to(device)
        with torch.no_grad():
            output = model(input)
        if i==0:
            targets=target
            outputs=output
            if return_image_paths==True:
                image_paths = image_path
        else:
            targets=torch.cat((targets,target))
            outputs=torch.cat((outputs,output),axis=0)
            if return_image_paths==True:
                image_paths += image_path
    if return_image_paths==True:
        return outputs, targets, image_paths
    return outputs, targets

def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)

def copy_sourcefile(output_dir, src_dir = 'src' ):    
    import os 
    import shutil
    import glob 
    source_dir = os.path.join(output_dir, src_dir)

    os.makedirs(source_dir, exist_ok=True)
    org_files1 = os.path.join('./', '*.py' )
    org_files2 = os.path.join('./', '*.sh' )
    org_files3 = os.path.join('./', '*.ipynb' )
    org_files4 = os.path.join('./', '*.txt' )
    org_files5 = os.path.join('./', '*.json' )    
    files =[]
    files = glob.glob(org_files1 )
    files += glob.glob(org_files2  )
    files += glob.glob(org_files3  )
    files += glob.glob(org_files4  ) 
    files += glob.glob(org_files5  )     

    # print("COPY source to output/source dir ", files)
    tgt_files = os.path.join( source_dir, '.' )
    for i, file in enumerate(files):
        shutil.copy(file, tgt_files)
class LossSaver(object):
    def __init__(self):
        self.train_losses = []
        self.val_losses = []
    def reset(self):
        self.train_losses = []
        self.val_losses = []
    def update(self, train_loss, val_loss):
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)
    def return_list(self):
        return self.train_losses, self.val_losses
    def save_as_csv(self, csv_file):
        df = pd.DataFrame({'Train Losses': self.train_losses, 'Validation Losses': self.val_losses})
        df.index = [f"{i+1} Epoch" for i in df.index]
        df.to_csv(csv_file, index=True)
class AverageMeter (object):
    def __init__(self):
        self.reset ()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
                
def collect_image_paths(base_dataset_dir, sub_dirs=['training', 'validation', 'test']):
    train_image_path_list = []
    train_target_path_list = []
    validation_image_path_list = []
    validation_target_path_list = []
    test_image_path_list = []
    test_target_path_list = []
    
    # 서브 디렉토리 이름과 리스트 매핑
    dir_to_lists = {
        'training': (train_image_path_list, train_target_path_list),
        'validation': (validation_image_path_list, validation_target_path_list),
        'test': (test_image_path_list, test_target_path_list)
    }
    
    for sub_dir in sub_dirs:
        # 각 서브 디렉토리의 전체 경로를 생성
        full_dir_path = os.path.join(base_dataset_dir, sub_dir)
        
        # 해당 서브 디렉토리에 대한 이미지 및 마스크 리스트 선택
        image_list, target_list = dir_to_lists.get(sub_dir, (None, None))
        
        if image_list is None or target_list is None:
            print(f"Unknown sub-directory: {sub_dir}")
            continue
        
        # 서브 디렉토리 내의 파일들을 검색
        for file_name in os.listdir(full_dir_path):
            # 이미지 파일이 .jpg 확장자이며 마스크 파일이 아닌지 확인
            if file_name.endswith(".jpg") and "_mask" not in file_name:
                # 이미지 파일의 전체 경로를 image_list에 추가
                image_list.append(os.path.join(full_dir_path, file_name))
                
                # 마스크 파일 이름 생성 (.jpg를 _mask.png로 변경)
                mask_file_name = file_name.replace(".jpg", "_mask.png")
                
                # 마스크 파일의 전체 경로를 target_list에 추가
                target_list.append(os.path.join(full_dir_path, mask_file_name))
    
    return (train_image_path_list, train_target_path_list,
            validation_image_path_list, validation_target_path_list,
            test_image_path_list, test_target_path_list)

def Do_Experiment(iteration, model_name, model, train_loader, validation_loader, test_loader, Optimizer, lr,  number_of_classes, epochs, Metrics,df,device, transform):
    start = timeit.default_timer()
    train_bool=True
    test_bool=True
    if loss_function == 'Tversky Focal Loss':
        criterion=TverskyLoss()
    elif loss_function == 'DiceBCELoss': ##원래 DiceBCELoss
        criterion=DiceBCELoss()
    if Optimizer=='Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif Optimizer == 'SGD':
        momentum = 0.9
        weight_decay = 1e-4
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay=weight_decay)
    elif Optimizer =='AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    if lr_scheduler_args['lr_scheduler'] == 'CosineAnnealingLR':
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = lr_scheduler_args['T_max'], eta_min = lr_scheduler_args['eta_min'])
    
    os.makedirs(output_dir, exist_ok = True)
    control_random_seed(seed)
    if train_bool:
        now = datetime.now()
        Train_date=now.strftime("%y%m%d_%H%M%S")
        print('Training Start Time:',Train_date)
        best=9999
        best_epoch=1
        Early_Stop=0
        loss_saver = LossSaver()
        train_start_time = timeit.default_timer()
        for epoch in range(1, epochs+1):
            Train_Loss = train(train_loader, epoch, 
              model, criterion, optimizer, device
              )
            lr_scheduler.step()
            outputs, targets  \
            = validate(validation_loader, 
              model, criterion, device
              )
            Val_Loss = np.round(criterion(outputs,targets).cpu().numpy(),6)            
            iou = np.round(Intersection_over_Union(outputs, targets),3)
            dice = np.round(Dice_Coefficient(outputs, targets),3)
            now = datetime.now()
            date=now.strftime("%y%m%d_%H%M%S")
            print(str(epoch)+'EP('+date+'):',end=' ')
            print('T_Loss: ' + str(Train_Loss), end=' ')
            print('V_Loss: ' + str(Val_Loss), end=' ')
            print('IoU: ' + str(iou), end=' ')
            print('Dice: ' + str(dice), end=' ')
            
            loss_saver.update(Train_Loss, Val_Loss)
            loss_saver.save_as_csv(f'{output_dir}/Losses_{Experiments_Time}.csv')
            if Val_Loss<best:
                Early_Stop = 0
                torch.save(model.state_dict(), f'{output_dir}/{Train_date}_{model_name}_Iter_{iteration}.pt')
                best_epoch = epoch
                best = Val_Loss
                print('Best Epoch:',best_epoch,'Loss:',Val_Loss)
            else:
                print('')
                Early_Stop+=1
            if Early_Stop>=EARLY_STOP:
                break
        train_stop_time = timeit.default_timer()
    if test_bool:
        now = datetime.now()
        date=now.strftime("%y%m%d_%H%M%S")
        print('Test Start Time:',date)
        outputs, targets, image_paths \
            = validate(test_loader, 
              model, criterion, device,
            model_path=f'{output_dir}/{Train_date}_{model_name}_Iter_{iteration}.pt',
                       return_image_paths=True
              )        
        Loss = np.round(criterion(outputs,targets).cpu().numpy(),6)
        pa = np.round(Pixel_Accuracy(outputs.cpu(), targets.cpu()),3)
        iou = np.round(Intersection_over_Union(outputs, targets),3)
        dice = np.round(Dice_Coefficient(outputs, targets),3)
        recall, precision, f1 = Confusion_Matrix(outputs, targets) 
        recall = np.round(recall.cpu().numpy()[0],3); precision = np.round(precision.cpu().numpy()[0],3); f1 = np.round(f1.cpu().numpy()[0],3);
                
        now = datetime.now()
        date=now.strftime("%y%m%d_%H%M%S")
        print('Best Epoch:',best_epoch)
        print('Test('+date+'): '+'Loss: ' + str(Loss),end=' ')
        print('PA: ' + str(pa), end=' ')
        print('IoU: ' + str(iou), end=' ')
        print('Dice: ' + str(dice), end=' ')
        print('Recall: ' + str(recall), end=' ')
        print('Precision: ' + str(precision), end=' ')
        print('F1 Score: ' + str(f1), end='\n')
                            
        stop = timeit.default_timer();m, s = divmod((train_stop_time - train_start_time)/epoch, 60);h, m = divmod(m, 60);Time_per_Epoch = "%02d:%02d:%02d" % (h, m, s);
        m, s = divmod(stop - start, 60);h, m = divmod(m, 60);Time = "%02d:%02d:%02d" % (h, m, s);
        total_params = sum(p.numel() for p in model.parameters()); total_params = format(total_params , ',');
        Performances = [Experiments_Time, Train_date, iteration, model_name, best, Loss, pa, iou, dice, recall, precision, f1, total_params,Time, best_epoch, Time_per_Epoch, loss_function, lr, batch_size, epochs, FILENAME]
        df = df.append(pd.Series(Performances, index=df.columns), ignore_index=True)
        os.makedirs(f'{output_dir}/test_outputs', exist_ok = True)
        outputs = outputs.cpu().numpy()
        for output, image_path in zip(outputs, image_paths):
            np.save(f'{output_dir}/test_outputs/{os.path.basename(image_path)}', output)
    now = datetime.now()
    date=now.strftime("%y%m%d_%H%M%S")
    print('End',date)
    
    return df

In [4]:
now = datetime.now()
Experiments_Time=now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)
Metrics=['Experiment Time','Train Time', 'Iteration','Model Name', 'Val_Loss', 'Test_Loss','PA', 'IoU', 'Dice', 'Recall', 'Precision', 'F1 Score', 'Total Params','Train-Predction Time','Best Epoch','Time per Epoch', 'Loss Function', 'LR', 'Batch size', '#Epochs', 'DIR']
df = pd.DataFrame(index=None, columns=Metrics)
output_root = f'output_proposedd/DeepLab_V3_Plus_Effi_U_Trans2(256)/output_{Experiments_Time}'
os.makedirs(output_root, exist_ok = True)

# iterations = [7, 8, 9, 10]

# for iteration in iterations:
# COCO 데이터셋 기반 반복 실험 코드
for iteration in range(iterations[0], iterations[1] + 1):
    seed = iteration
    control_random_seed(seed)
    
    # 각 iteration에 맞는 split 폴더 설정
    Dataset_dir = f'CarDD_release/splits/split{str(iteration).zfill(2)}'
    
    # COCO 데이터 로드
    coco_data = load_coco_data(Dataset_dir)
    
    # 데이터 로더 생성
    train_loader = create_dataloader(
        coco_data, "training", os.path.join(Dataset_dir, "training"), batch_size, aug=True
    )
    validation_loader = create_dataloader(
        coco_data, "validation", os.path.join(Dataset_dir, "validation"), batch_size, aug=False
    )
    test_loader = create_dataloader(
        coco_data, "test", os.path.join(Dataset_dir, "test"), batch_size, aug=False
    )

    # 반복 실험 실행
    for model_name in model_names:
        print(f'{model_name} (Iter {iteration})')
        output_dir = output_root + f'/{model_name}_Iter_{iteration}'
        copy_sourcefile(output_dir, src_dir='src')
        control_random_seed(seed)

        # 모델 초기화
        model = str_to_class(model_name)(in_channels, number_of_classes)
        device_ids = [2]
        device = torch.device("cuda:" + str(device_ids[0]))
        if len(device_ids) > 1:
            model = torch.nn.DataParallel(model, device_ids=device_ids).to(device)
        else:
            model = model.to(device)

        # 실험 수행
        df = Do_Experiment(
            seed, model_name, model, train_loader, validation_loader, test_loader,
            optimizer, lr, number_of_classes, epochs, Metrics, df, device, None
        )

        # 결과 저장
        try:
            df.to_csv(output_root + '/' + f'output_CarDD_DeepLab_V3_Plus_Effi_U_Trans2(256)_{Experiments_Time}.csv', index=False, header=True, encoding="cp949")
        except:
            now = datetime.now()
            tmp_date = now.strftime("%y%m%d_%H%M%S")
            df.to_csv(output_root + '/' + f'output_CarDD_DeepLab_V3_Plus_Effi_U_Trans2(256)_{Experiments_Time}_{tmp_date}_tmp.csv', index=False, header=True, encoding="cp949")

print('End')
os._exit(0)

Experiment Start Time: 250411_222419
DeepLab_V3_Plus_Effi_U_Trans2 (Iter 1)
2025-04-11 22:24:23,347 - Loading pretrained weights from Hugging Face hub (timm/efficientnet_b5.sw_in12k_ft_in1k)
2025-04-11 22:24:23,885 - [timm/efficientnet_b5.sw_in12k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-04-11 22:24:28,642 - Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
Training Start Time: 250411_222444
1EP(250411_222607): T_Loss: 0.995703 V_Loss: 0.740364 IoU: 0.54 Dice: 0.648 Best Epoch: 1 Loss: 0.740364
2EP(250411_222636): T_Loss: 0.772316 V_Loss: 0.639074 IoU: 0.529 Dice: 0.633 Best Epoch: 2 Loss: 0.639074
3EP(250411_222707): T_Loss: 0.641473 V_Loss: 0.541569 IoU: 0.591 Dice: 0.69 Best Epoch: 3 Loss: 0.541569
4EP(250411_22

KeyboardInterrupt: 